# Introducción al pre-procesado de texto para Procesamiento de Lenguaje Natural 
[Pablo Carballeira] Partes de este código han sido adaptadas del código correspondiente a la especialización Natural Language Processing de DeepLearning.AI

Puedes encontrar información sobre cómo trabajar en Colab aquí (https://colab.research.google.com/notebooks/intro.ipynb)



El preprocesamiento de datos es uno de los pasos críticos en cualquier proyecto de data science. Incluye limpiar y formatear los datos antes de introducirlos en un algoritmo de aprendizaje automático. Es fundamental en la mayoría de casos en PLN, y  los pasos de preprocesamiento suelen estar compuestos de las siguientes tareas:

* Segmentación en palabras o tokenización
* Conversión a minúsculas
* Eliminación de palabras supérfluas (stopwords en inglés) y puntuación
* Normalización (radicalización y/o lematización)

En este notebook, y en varios de los siguientes, usaremos el paquete [Natural Language Toolkit (NLTK)](http://www.nltk.org/howto/twitter.html), una biblioteca Python de código abierto para el procesamiento del lenguaje natural.

Proporciona una interfaz sencilla de utilizar, con más de 50 corpus y recursos léxicos como WordNet, junto con un conjunto de bibliotecas de procesamiento de texto para clasificación, tokenización, lematización, etiquetado, o análisis semántico. 

El libro [Natural Language Processing with Python](https://www.nltk.org/book/) proporciona una introducción práctica a la programación para el procesamiento del lenguaje. Escrito por los creadores de NLTK, guía al lector a través de los fundamentos de la escritura de programas Python, el trabajo con corpus, la categorización del texto, el análisis de la estructura lingüística y más. 

In [ ]:
# Importamos y descargamos algunos paquetes necesarios
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('treebank')

from nltk.tree import Tree
from IPython.display import display

import os

## Dataset de Twitter

Antes de empezar, vamos a probar un conjunto de datos de Twitter, que viene integrado en NLTK, para hacernos una idea de la necesidad de este pre-procesado. Además, NLTK tiene módulos para recopilar, manejar y procesar datos de Twitter. Es un conjunto de datos anotado manualmente (etiquetas e opinión positiva y negativa) y es útil para medir de manera rápida el rendimiento base de algoritmos de clasificación de texto, que veremos más adelante.


In [ ]:
from nltk.corpus import twitter_samples    # sample Twitter dataset from NLTK
import matplotlib.pyplot as plt            # library for visualization
import random                              # pseudo-random number generator

nltk.download('twitter_samples')

Podemos cargar los textos de los tweets positivos y negativos usando el método `strings()` del módulo de esta manera:

In [ ]:
# select the set of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

A continuación, imprimiremos el número de tweets positivos y negativos. Así como la estructura de datos del dataset. 



In [ ]:
print('Number of positive tweets: ', len(all_positive_tweets))
print('Number of negative tweets: ', len(all_negative_tweets))

print('\nThe type of all_positive_tweets is: ', type(all_positive_tweets))
print('The type of a tweet entry is: ', type(all_negative_tweets[0]))

## Texto de los tweets sin preprocesar

Antes que nada, podemos imprimir un par de tweets del conjunto de datos para ver cómo son. El conocimiento de los datos es responsable de una gran parte del éxito o fracaso de los proyectos de ciencia de datos. Podemos observar aspectos que debemos tener en cuenta al preprocesar nuestros datos.

A continuación, se imprimirá un tweet positivo al azar y otro negativo al azar. Hemos agregado una marca de color al comienzo de la cadena para distinguirlos aún más. (Advertencia: esta base de datos está tomada de un conjunto de datos públicos de tweets reales y una porción muy pequeña tiene contenido explícito). Puedes ejecutar varias veces la celda para ver varios ejemplos.

In [ ]:
# print positive in greeen
print('\033[92m' + all_positive_tweets[random.randint(0,5000)])

# print negative in red
print('\033[91m' + all_negative_tweets[random.randint(0,5000)])

Se puede observar la presencia de [emoticonos](https://en.wikipedia.org/wiki/Emoticon) y URLs en muchos de los tweets. Conocer esta información será útil en los siguientes pasos.

Utilizaremos el siguiente tweet para ejemplificar los posibles pasos del pre-procesado de texto

In [ ]:
# Our selected sample. Complex enough to exemplify each step
tweet = all_positive_tweets[2277]
print(tweet)

## Eliminar hipervínculos, marcas y estilos de Twitter

Dado que tenemos un conjunto de datos de Twitter, nos gustaría eliminar algunas subcadenas que se usan comúnmente en la plataforma, como el hashtag, las marcas de retweet y los hipervínculos. Usaremos la biblioteca de expresiones regulares [re](https://docs.python.org/3/library/re.html) para realizar operaciones basadas en expresiones regulares en nuestro tweet.

Definiremos nuestro patrón de búsqueda y usaremos el método `sub()` para eliminar las coincidencias sustituyéndolas con un carácter vacío (es decir, `''`)

In [ ]:
import re                                  # library for regular expression operations

print('\033[92m' + tweet)
print('\033[94m')

# remove old style retweet text "RT"
tweet2 = re.sub(r'^RT[\s]+', '', tweet)

# remove hyperlinks
tweet2 = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet2)

# remove hashtags
# only removing the hash # sign from the word
tweet2 = re.sub(r'#', '', tweet2)

print(tweet2)

## Tokenización

La segmentación o "tokenización" de texto es una de las herramientas fundamentales para preprocesar textos de lenguaje natural. En este notebook vamos a ver algunas de las aproximaciones mas comunes, utilizando en este caso el lenguaje inglés como ejemplo

Una de las maneras mas sencillas de realizar esta tarea es segmentar los tokens (palabras) separados por espacios en blanco

In [ ]:
text = "This is Andrew's text, isn't it?"
tokenizer = nltk.tokenize.WhitespaceTokenizer()
tokenizer.tokenize(text)


Uno de los problemas de este método es que puede obtener tokens diferentes para palabras con el mismo significado. Por ejemplo el "it?" del final de la frase, comparado con un "it" en otro lugar de una frase

Otra de las posibilidades es tokenizar mediante la separación establecida por los signos de puntuación. El problema de este método es que puede sobresegmentar la oración en tokens sin significado completo

In [ ]:
tokenizer = nltk.tokenize.WordPunctTokenizer()
tokenizer.tokenize(text)

Existen métodos más avanzados que tienen en cuenta relaciones más complejas, obtenidas ya sea mediante reglas o mediante aprendizaje. En este caso utilizamos un método basado en reglas del lenguaje incluido en la librería de NLTK

In [ ]:
tokenizer = nltk.tokenize.TreebankWordTokenizer()
print(tokenizer.tokenize(text))
# el mismo método se puede invocar con la funcion word_tokenize()
print(word_tokenize(text))

Dependiendo del idioma, este proceso puede no ser evidente, ni codificable según reglas fijas. Por ejemplo en el caso de idiomas con abundancia de palabras compuestas, como el alemán. 

Rechtsschutzversicherungsgesellschaften: compañía de seguros que proporciona protección legal 

Fijémonos que la segmentación no es correcta, incluso itentando utilizar reglas aplicadas al idioma aleman

In [ ]:
text = "Rechtsschutzversicherungsgesellschaften"
print("\nOriginal string:")
print(text)
from nltk.tokenize import word_tokenize
token_text = word_tokenize(text, language='german')
print("\nWord-tokenized copy in a list:")
print(token_text)

NLTK nos proprciona un método TweetTokenizer, que funciona de forma muy similar al método mas genérico [word_tokenize](https://www.nltk.org/api/nltk.tokenize.html#module-nltk.tokenize.casual). TweetTokenizer mantiene intactos los hashtags mientras que word_tokenize no lo hace.

Fijémonos en cuales son los argumentos opcionales de TweetTokenizer, y como los usamos en el ejemplo para convertir el texto a minúsculas y eliminar posibles repeticiones de caracteres.

- preserve_case (bool) – Flag indicating whether to preserve the casing (capitalisation) of text used in the tokenize method. Defaults to True.

- reduce_len (bool) – Flag indicating whether to replace repeated character sequences of length 3 or greater with sequences of length 3. Defaults to False.

- strip_handles (bool) – Flag indicating whether to remove Twitter handles of text used in the tokenize method. Defaults to False.

In [ ]:
from nltk.tokenize import TweetTokenizer

print('\033[92m' + tweet)
print()
print('\033[92m' + tweet2)
print('\033[94m')

# instantiate tokenizer class
tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)

# tokenize tweets
tweet_tokens = tokenizer.tokenize(tweet2)

print()
print('Tokenized string:')
print(tweet_tokens)

## Eliminar palabras vacías y signos de puntuación

En muchos casos, es útil eliminar  las palabras vacías y la puntuación. Las palabras vacías (stopwords en inglés) son palabras que no agregan un significado relevante al texto. Verá la lista proporcionada por NLTK cuando ejecutes la celdas a continuación.

In [ ]:
# download the stopwords from NLTK
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
import string

#Import the english stop words list from NLTK
stopwords_english = stopwords.words('english') 

print('Stop words\n')
print(stopwords_english)

print('\nPunctuation\n')
print(string.punctuation)

Podemos ver que la lista de palabras vacías anterior contiene algunas palabras que podrían ser importantes en algunos contextos.
Estas podrían ser palabras como _i, not, between, because, won, against_. Es posible que debas personalizar la lista de palabras vacías para algunas aplicaciones. Para nuestro ejercicio, usaremos la lista completa.

Para la puntuación, vimos anteriormente que ciertas agrupaciones como ':)' y '...' deben conservarse cuando se trata de tweets porque se usan para expresar emociones. En otros contextos, estos también deben eliminarse.

¡Es hora de limpiar nuestro tweet tokenizado!

### Ejercicio

Completar el bucle, añadiendo a tweets_clean aquellos tokens que no estén incluidos en la lista de stpowords o puntuación

In [ ]:
print('\033[92m' + tweet)
print()
print('\033[92m')
print(tweet_tokens)
print('\033[94m')

tweets_clean = []

for word in tweet_tokens: # Go through every word in your tokens list
    # tweets_clean = ???

print('removed stop words and punctuation:')
print(tweets_clean)

Fíjate que las palabras **happy** y **sunny** en esta lista están escritas correctamente.

## Normalización

En ciertos casos puede ser interesante normalizar los tokens (palabras) para obtener un único token a partir de diferentes versiones de una palabra, p.ej wolf, wolves --> wolf, o talk, talks --> talk. 

Esto reduce el vocabulario que es necesario considerar en el caso de una aplicación de análisis de opinión por ejemplo.

### Radicalización

Radicalización o derivación (stemming): consiste en la eliminación de sufijos, convirtiéndola en su forma más general. Uno de los modelos mas comunes de radicalización es el modelo de Porter, basado en reglas morfoloógicas.


Considerando las palabras: 
 * **learn**
 * **learn**ing
 * **learn**ed
 * **learn**t
 
Todas estas palabras se derivan de su raíz común **learn**. Sin embargo, en algunos casos, el proceso de lematización produce palabras que no son ortografías correctas de la raíz de la palabra. Por ejemplo, **happi** y **sunni**. Eso es porque elige la raíz más común para las palabras relacionadas. Por ejemplo, podemos fijarnos en el conjunto de palabras que componen las distintas formas de happy:

 * **happ**y
 * **happi**ness
 * **happi**er
 
Podemos ver que el prefijo **happi** se usa más comúnmente. No podemos elegir **happ** porque es la raíz de palabras no relacionadas como **happen**
 





In [ ]:
text = "feet wolves cats talked"
tokenizer = nltk.tokenize.TreebankWordTokenizer()
tokens = tokenizer.tokenize(text)
stemmer = nltk.stem.PorterStemmer()
" ".join(stemmer.stem(token) for token in tokens)

Vemos como es común generar errores en formas irregulares, como foot --> feet

### Lematización
 
La lematización consiste en una tarea similar pero con la utilización de vocabulario y análisis morfológico. Trata recupurar la forma básica de una palabra (o versión del diccionario). En el ejemplo se utiliza un lematizador basado en la base de datos WordNet (que codifica relaciones entre palabras)



In [ ]:
stemmer = nltk.stem.WordNetLemmatizer()
" ".join(stemmer.lemmatize(token) for token in tokens)


La lematización que hemos utilizado puede obtener resultados no deseados en el caso de verbos, ya que asume por defecto que todas las palabras son sustantivos.

Mira lo que ocurre si indicamos a la función que las palabras son verbos



 


In [ ]:
" ".join(stemmer.lemmatize(token,"v") for token in tokens)


En este caso, la lematización del verbo es correcta, pero no así los sustantivos. Por lo tanto, sería interesante usar etiquetas morfológicas para hacer esta lematización de forma más inteligente. 

Veremos herramientas para hacer esto, de forma automática, más adelante 

## Radicalización de tuits

NLTK tiene diferentes módulos para radicalización. En el caso de nuestros datos de Twitter, utilizaremos la radicalización de Porter [PorterStemmer](https://www.nltk.org/api/nltk.stem.html#module-nltk.stem.porter) que usa el [Algoritmo de radicalización de Porter](https://tartarus.org/martin/PorterStemmer/).

In [ ]:
from nltk.stem import PorterStemmer        # module for stemming

print('\033[92m' + tweet)
print()
print('\033[92m')
print(tweets_clean)
print('\033[94m')

# Instantiate stemming class
stemmer = PorterStemmer() 

# Create an empty list to store the stems
tweets_stem = [] 

for word in tweets_clean:
    stem_word = stemmer.stem(word)  # stemming word
    tweets_stem.append(stem_word)  # append to the list

print('stemmed words:')
print(tweets_stem)

## Función para el pre-procesado del dataset completo

Una vez que hemos visto las tareas típicas de preprocesado, vamos a juntarlo todo en una única función que podamos utilizar para preprocesar nuestro dataset de tweets completo.

### Ejercicio

Completa la funcion `process_tweet(tweet)`, utilizando los pasos que hemos definido más arriba

Ejecutando el código de la siguiente celda puedes comprobar que el resultado es el mismo que haciendo el pre-procesado paso a paso


In [ ]:
import re
import numpy as np

from nltk.stem import PorterStemmer


def process_tweet(tweet):
    """Process tweet function.
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    """
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    
    # incluir la eliminacion de hyperlinks, etc que hemos visto antes
    # tweet = ???
    
    # tokenización
    # tweet_tokens = ???

    # eliminación de stopwords, puntuación y radicalización
    # realizar todo a la vez en el mismo bucle
    tweets_clean = []

    # tweets_clean = ???

    return tweets_clean

In [ ]:
# choose the same tweet
tweet = all_positive_tweets[2277]

print()
print('\033[92m')
print(tweet)
print('\033[94m')

# call the imported function
tweets_stem = process_tweet(tweet); # Preprocess a given tweet

print('preprocessed tweet:')
print(tweets_stem) # Print the result

### Ejercicio

Puedes ejecutar la próxima celda múltiples veces para comprobar cual es el resultado de pre-procesar diferentes tweets de nuestro dataset

In [ ]:
# choose the same tweet
tweet_pos = all_positive_tweets[random.randint(0,5000)]
tweet_neg = all_negative_tweets[random.randint(0,5000)]

# call the preprocessing function
tweet_pos_stem = process_tweet(tweet_pos); # Preprocess
tweet_neg_stem = process_tweet(tweet_neg); # Preprocess

print()
print('\033[92m')
print(tweet_pos)
print('\033[94m')

print('preprocessed tweet:')
print(tweet_pos_stem) # Print the result

print()
print('\033[91m')
print(tweet_neg)
print('\033[94m')

print('preprocessed tweet:')
print(tweet_neg_stem) # Print the result